<a href="https://colab.research.google.com/github/tguinot/difiq/blob/main/Local_vol_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo dans un modèle à volatilité locale simplifié

> **Antonin Chaix**

La dynamique de l'actif est donné par l'EDS suivante, sous la mesure risque-neutre :

$$
\frac{dS_t}{S_t}=\mu(t) dt+\sigma(t,S_t)dW_t
$$

On s'intéresse à ce modèle dans le cas d'un actif sans dividendes ($\mu(t)=r$) et d'une nappe de volatilité implicite très simple, donnée par :

$$
\hat \sigma(T,K) = a + b (K-S_0)
$$

avec `a = 0.2` et `b = -0.001`.

**NB** : une version beaucoup plus générale du modèle à vol locale est implémentée dans [ce colab](https://colab.research.google.com/drive/1Jc0EUawS9uXigJ7A4er1joMr-8eJH80W?usp=sharing).

Rappelons la formule de Dupire :

$$
\sigma^2(T,K)=\frac{2\frac{\partial \hat\sigma}{\partial T}+\frac{\hat\sigma}{T}+2K\mu(T)\frac{\partial \hat\sigma}{\partial K}}
{K^2\left[\frac{\partial^2 \hat\sigma}{\partial K^2}-d_1\sqrt{T}\left(\frac{\partial \hat\sigma}{\partial K}\right)^2+\frac{1}{\hat\sigma
}\left(\frac{1}{K\sqrt{T}}+d_1\frac{\partial \hat\sigma}{\partial K}\right)^2\right]}\\
$$
avec
$$
d_1=\frac{\ln(S_0/K)+\int_0^T\mu(s)ds}{\hat\sigma\sqrt{T}}
+ \frac{1}{2}\hat\sigma\sqrt{T}
$$

Elle se simplifie ici en :

$$
\sigma^2(T,K)=\frac{\frac{\hat\sigma}{T}+2Krb}
{K^2\left[-d_1\sqrt{T}b^2+\frac{1}{\hat\sigma
}\left(\frac{1}{K\sqrt{T}}+d_1b\right)^2\right]}
$$

Pour obtenir la volatililité au point $(t,S_t)$, on calculera donc :

$$
\sigma(t,S_t)=\sqrt{\frac{\frac{\hat\sigma(t,S_t)}{t}+2S_trb}
{S_t^2\left[-d_1\sqrt{t}b^2+\frac{1}{\hat\sigma(t,S_t)
}\left(\frac{1}{S_t\sqrt{t}}+d_1b\right)^2\right]}}
$$

avec :

$$
\hat \sigma(t,S_t) = a + b (S_t-S_0)\qquad\qquad d_1=\frac{\ln(S_0/S_t)+rt}{\hat\sigma\sqrt{t}}
+ \frac{1}{2}\hat\sigma\sqrt{t}
$$

On définit tout d'abord vol implicite et vol locale :

In [ ]:
import numpy as np
import math

# params vol implicite
A = 0.2
B = -0.001

def implied_vol(T, K) :
	return A + B * (K - S0)

def local_vol(t, S) :
	if (t==0) : t = 0.0000000001
	sqrt_t = np.sqrt(t)
	vol = implied_vol(t, S)
	d1 = (np.log(S0 / S) + (r + 0.5 * vol**2) * t) / (vol * sqrt_t)
	denom = S**2 * (-d1 * sqrt_t * B**2 + (1 / (S * sqrt_t) + d1 * B)**2 / vol)
	return np.sqrt((vol / t + 2 * S * r * B) / denom)


Puis on génère les trajectoires de l'actif dans le modèle à vol locale.

In [ ]:
# Horizon du Monte Carlo = maturité des options évaluées
T = 1

# taux d'intérêt
r = 0.015

# spot
S0 = 100

# MC params
Nsimul = 5*10**5
Nsteps = 100

# time steps
dT = T / Nsteps
t = np.linspace(0, T, num=Nsteps+1, endpoint=True)

# generation normales
norm = np.random.normal(0, 1, (Nsteps, Nsimul))

# initialisation du tableau contenant les trajectoires
S = np.empty(shape=(Nsteps + 1, Nsimul))
S[0,:] = S0

# precalc
sqrt_dt = math.sqrt(dT)

# MC loop
for j in range(Nsteps) :
	sigma = local_vol(t[j], S[j,:])
	S[j+1,:] = S[j,:] * np.exp( (r - 0.5 * sigma**2) * dT + sigma * sqrt_dt * norm[j,:])


Il n'y a plus qu'à évaluer les options souhaitées et à comparer les prix obtenus aux valeurs attendues...

In [ ]:
from scipy.stats import norm

# BS call function
def bs_call(T, K, F0, sigma, r) :
	sigma_sqrt_T = sigma * math.sqrt(T)
	d1 = (math.log(F0/K) + (r + 0.5 * sigma**2) * T) / sigma_sqrt_T
	d2 = d1 - sigma_sqrt_T
	return S0 * norm.cdf(d1) - K * math.exp(-r*T) * norm.cdf(d2)

# MC price & confidence interval
def compute_and_print_price(payoff, option_name, closed_form_price = -1) :
	price = np.mean(payoff)
	stdev = np.std(payoff)
	IClow = price - 1.96 * stdev / math.sqrt(Nsimul)
	ICup = price + 1.96 * stdev / math.sqrt(Nsimul)
	if (closed_form_price == -1) : print("{} = {:.4f}".format(option_name, price) )
	else : 	print("{} = {:.4f} (closed-form price = {:.4f})".format(option_name, price, closed_form_price) )
	print("IC 95% = [{:.4f} ; {:.4f}]".format(IClow,ICup) )
	print("-----------------------------------------------------")

# strikes
K_atm = 100
K_itm = 80
K_otm = 120

# down & out call barrier
H = 90

# discount factor
DF = math.exp(-r * T)

# MC payoffs
payoff_call_atm = DF * np.maximum(S[-1,:] - K_atm , 0)
payoff_call_itm = DF * np.maximum(S[-1,:] - K_itm , 0)
payoff_call_otm = DF * np.maximum(S[-1,:] - K_otm , 0)
payoff_call_down_and_out = np.where(S.min(axis=0) > H, payoff_call_atm, 0.)
payoff_digital = DF * np.where(S[-1,:] > K_atm, 1., 0.)

# closed forms
call_atm = bs_call(T, K_atm, S0, implied_vol(T, K_atm), r)
call_itm = bs_call(T, K_itm, S0, implied_vol(T, K_itm), r)
call_otm = bs_call(T, K_otm, S0, implied_vol(T, K_otm), r)

vol_minus = implied_vol(T, K_atm-1)
vol_plus  = implied_vol(T, K_atm+1)
digital_atm = (bs_call(T, K_atm-1, S0, vol_minus, r) - bs_call(T, K_atm+1, S0, vol_plus, r)) / 2.
digital_atm_bs = (bs_call(T, K_atm-1, S0, implied_vol(T, K_atm), r) - bs_call(T, K_atm+1, S0, implied_vol(T, K_atm), r)) / 2.

# compute & display
print("ATM vol = {:.2f}%".format(100*implied_vol(T, K_atm)))
compute_and_print_price(payoff_call_atm, "Call ATM vol locale", call_atm)
print("ITM vol = {:.2f}%".format(100*implied_vol(T, K_itm)))
compute_and_print_price(payoff_call_itm, "Call ITM vol locale", call_itm)
print("OTM vol = {:.2f}%".format(100*implied_vol(T, K_otm)))
compute_and_print_price(payoff_call_otm, "Call OTM vol locale", call_otm)
compute_and_print_price(payoff_call_down_and_out, "Call Down & Out vol locale")
compute_and_print_price(payoff_digital, "Binaire ATM vol locale", digital_atm)
print("Binaire ATM BS (no skew) = {:.4f}".format(digital_atm_bs))


ATM vol = 20.00%
Call ATM vol locale = 8.6733 (closed-form price = 8.6728)
IC 95% = [8.6404 ; 8.7063]
-----------------------------------------------------
ITM vol = 22.00%
Call ITM vol locale = 22.5707 (closed-form price = 22.5729)
IC 95% = [22.5225 ; 22.6188]
-----------------------------------------------------
OTM vol = 18.00%
Call OTM vol locale = 1.8684 (closed-form price = 1.8566)
IC 95% = [1.8537 ; 1.8831]
-----------------------------------------------------
Call Down & Out vol locale = 7.2840
IC 95% = [7.2513 ; 7.3167]
-----------------------------------------------------
Binaire ATM vol locale = 0.5215 (closed-form price = 0.5220)
IC 95% = [0.5202 ; 0.5229]
-----------------------------------------------------
Binaire ATM BS (no skew) = 0.4828
